## Importing Nltk library

In [1]:
import nltk

## Importing other required modules

In [ ]:
import pandas as pd

In [ ]:
fake = pd.read_csv("Fake.csv")
genuine = pd.read_csv("True.csv")

### Printing Info

In [ ]:
print(fake.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23481 entries, 0 to 23480
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    23481 non-null  object
 1   text     23481 non-null  object
 2   subject  23481 non-null  object
 3   date     23481 non-null  object
dtypes: object(4)
memory usage: 733.9+ KB
None


In [ ]:
print(genuine.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21417 entries, 0 to 21416
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    21417 non-null  object
 1   text     21417 non-null  object
 2   subject  21417 non-null  object
 3   date     21417 non-null  object
dtypes: object(4)
memory usage: 669.4+ KB
None


In [ ]:
fake['target']=0
genuine['target']=1

In [ ]:
genuine.head()

,title,text,subject,date,target
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [ ]:
fake.head()

,title,text,subject,date,target
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


## Concatenating both datasets

In [ ]:
df=pd.concat([fake,genuine],axis=0) 
df=df.reset_index(drop=True)

Dropping unnecessary columns

In [ ]:
df=df.drop(['subject','date','title'],axis=1)
df.columns

Index(['text', 'target'], dtype='object')

### Preprocessing data

1. Tokenizing words

In [ ]:
from nltk.tokenize import word_tokenize
df['text']=df['text'].apply(word_tokenize)
df

2. Stemming Words

In [ ]:
from nltk.stem.snowball import SnowballSteamer
ss=SnowballSteamer("english")

In [ ]:
def stem_word(text):
    return[ss.stem(word) for word in text]

In [ ]:
df['text']=df['text'].apply(stem_word)
df

### Stopword Removal

In [ ]:
from nltk.corpus import stopwords

In [ ]:
def rem_stopwords(text):
    dt=[word for word in text if len(word)>2]
    return dt

In [ ]:
df['text']=df['text'].apply(rem_stopwords)
df

### Join text

In [ ]:
df['text']=df['text'].apply(' '.join)

## Splitting to Train and test data

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df['text'],df['target'])

## Vectorization

In [ ]:
from sklearn.feature_extraction.text import IfidfVectorizer
Iv= IfidfVectorizer(max_df=0.7)

In [ ]:
X_train = Iv.fit_transform(X_train)
X_test=Iv.transform(X_test)

In [ ]:
X_train.head()

In [ ]:
X_test.head()

## Training model with Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
m=LogisticRegression(max_iter=50)
m.fit(X_train,y_train)
prediction = m.predict(X_test)
print((accuracy_score(y_test,prediction))*100)